<center><img src='https://blog.gdeltproject.org/wp-content/uploads/2015-gdelt-2.png' width=400></center>

<strong>
Valentin LARRIEU <br>
Arnaud LEJEUNE <br>
Thomas MEIMOUN <br>
Thibault ROYER <br>
David TANG <br>
</strong>

In [10]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.son import SON
from pymongo import MongoClient
import time
import conda
import os
import seaborn as sns
sns.set(style='whitegrid')

#BaseMap
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib
from mpl_toolkits.basemap import Basemap

In [11]:
start = time. time()

In [12]:
client = MongoClient('mongodb://localhost:8157/')

In [13]:
db = client['gdelt']
collection = db.actor

<Strong>Fonction de Rename </strong>

In [14]:
path_country_code = "corresp_pays.csv"

In [15]:
df_country_code = pd.read_csv(path_country_code, low_memory=False, sep=",", dtype='object')

In [16]:
df_country_code = df_country_code.rename(columns={"code": 'Actor'}) # we rename the column for the join

In [17]:
def replace_by_correspondence(df, df_corresp, column_shared, index_column_replacer = 1, replace_nan = True):
    column_replacer = df_corresp.columns[index_column_replacer]
    dictionary = df_corresp.set_index(column_shared).to_dict()[column_replacer]
    df[column_shared + '_corresp'] = df[column_shared].map(dictionary)
    # We replace nan values by nothing
    if (replace_nan):
        df[column_shared + '_corresp'] = df[column_shared + '_corresp'].fillna("")
    return df

In [18]:
df_country_code.head()

,Actor,countryName
0,AF,Afghanistan
1,AX,Aland Islands
2,AL,Albania
3,AG,Algeria
4,AQ,American Samoa


# Part 1.

display the number of articles/events there were for each triplet (day, country of the event, language of the article).

In [19]:
def Function1(correspondance=1):
    
    pipeline = [
    {"$group": {"_id":{"jour":"$SQLDATE", "actor":"$ActionGeo_CountryCode",
                       "langue":"$MentionDocTranslationInfo"}, "count": {"$sum":1}}}    
    ]
    
    result_query = pd.DataFrame(list(collection.aggregate(pipeline, allowDiskUse = True)))
    DF = pd.DataFrame()

    DF['Jour'] = pd.to_datetime(result_query.apply(lambda row : row["_id"]["jour"] , axis = 1), format='%Y%m%d')
    DF['Actor'] = result_query.apply(lambda row : row["_id"]["actor"] , axis = 1)
    DF['Langue'] = result_query.apply(lambda row : row["_id"]["langue"] , axis = 1)
    DF['Count'] = result_query['count']
    
    if correspondance == 1:
        DF=replace_by_correspondence(DF, df_country_code, 'Actor')
    
    return DF

def Triplet_Q1(Date, Actor, Langue, DF):

    return DF.loc[(DF["Actor"]==Actor) & (DF['Langue'] == Langue) & (DF['Jour'] == Date)].style.hide_index()

def Function1_count(jourTest, paysTest, langueTest):
    """sort uniquement le count pour un triplet"""
    
    pipeline = [
   {"$match": {"$and":[ {"SQLDATE": jourTest}, {"ActionGeo_CountryCode": paysTest},
                       {"MentionDocTranslationInfo": langueTest} ]}},
   {"$group": { "_id":{"jour":"$SQLDATE", "pays":"$ActionGeo_CountryCode",
                       "langue":"$MentionDocTranslationInfo"}, "count": {"$sum":1}}} 
    ]
    
    DF = pd.DataFrame(list(collection.aggregate(pipeline))).style.hide_index()
    
    return DF

In [42]:
DF_Q1 = Function1()
DF_Q1.sort_values(by=['Count'], ascending = False).head(10).style.hide_index()

Jour,Actor,Langue,Count,Actor_corresp
2018-11-07 00:00:00,US,ENG,238124,United States
2018-03-14 00:00:00,US,ENG,205550,United States
2018-03-01 00:00:00,US,ENG,194166,United States
2018-04-04 00:00:00,US,ENG,193670,United States
2018-02-28 00:00:00,US,ENG,191072,United States
2018-04-19 00:00:00,US,ENG,191038,United States
2018-04-18 00:00:00,US,ENG,190252,United States
2018-03-07 00:00:00,US,ENG,190134,United States
2018-11-08 00:00:00,US,ENG,188432,United States
2018-06-28 00:00:00,US,ENG,187434,United States


In [43]:
DF_Q1.to_csv('DF_Q1.csv')

In [ ]:
Triplet_Q1("2018-02-15", "US", "ENG", DF_Q1)

Jour,Actor,Langue,Count,Actor_corresp
2018-02-15 00:00:00,US,ENG,182420,United States


In [23]:
Dataf = Function1_count(20180215, "US",  "ENG")
Dataf

_id,count
"{'jour': 20180215, 'pays': 'US', 'langue': 'ENG'}",182420


# Part 2.

for an actor given as a parameter, display the events (values in the events table) that refer to it (in the last 6 months).

In [20]:
def Function2(TheName, Date=20180601):
    """Méthode simple poure requeter la Q2"""
    
    pipeline = [
    { 
        "$match" : { "$and": [{"SQLDATE":{"$gte":Date}}, {"ActorName" : TheName} ]}
    },
    { 
        "$group" : { 
            "_id" : {"globalEventId":"$GLOBALEVENTID", "date":"$SQLDATE", "actor":"$ActorName"}
        }
    }
    ]
    
    result_query = pd.DataFrame(list(collection.aggregate(pipeline, allowDiskUse = True)))
    DF = pd.DataFrame()
    
    DF['Date'] = pd.to_datetime(result_query.apply(lambda row : row["_id"]["date"] , axis = 1), format='%Y%m%d')
    DF['EventID'] = result_query.apply(lambda row : row["_id"]["globalEventId"] , axis = 1)
    DF['Actor'] = result_query.apply(lambda row : row["_id"]["actor"] , axis = 1)    
    
    return DF

In [22]:
DF_Q2 = Function2("BARACK OBAMA")
DF_Q2.sort_values(by=["Date"], ascending = False).head(10).style.hide_index()

Date,EventID,Actor
2018-12-31 00:00:00,813286748,BARACK OBAMA
2018-12-31 00:00:00,813377065,BARACK OBAMA
2018-12-31 00:00:00,813272021,BARACK OBAMA
2018-12-31 00:00:00,813361802,BARACK OBAMA
2018-12-31 00:00:00,813357513,BARACK OBAMA
2018-12-31 00:00:00,813298033,BARACK OBAMA
2018-12-31 00:00:00,813286729,BARACK OBAMA
2018-12-31 00:00:00,813258141,BARACK OBAMA
2018-12-31 00:00:00,813357508,BARACK OBAMA
2018-12-31 00:00:00,813253259,BARACK OBAMA


In [33]:
DF_Q2.to_csv('DF_Q2.csv')

# Part 3

find the subjects (actors) who had the most positive/negative articles for each triplet (month, country, language of the article).

In [21]:
def Function3():
    pipeline = [    
    {
        "$project": {
            "MonthYear": 1, "ActorName" : 1, "MentionDocTranslationInfo" : 1, "ActorGeo_CountryCode":1 ,
            "lessThan0": {  #// Set to 1 if AvgTone < 0
                "$cond": [ { "$lt": [ "$AvgTone", 0 ] }, 1, 0] #// MentionDocTone vs AvgTone ! 
            },
            "moreThan0": {  #// Set to 1 if AvgTone > 0
                "$cond": [ { "$gt": [ "$AvgTone", 0 ] }, 1, 0]
            }
        }
    },
    {
        "$group": {
            "_id": {"mois": "$MonthYear",  "actor":"$ActorName","langue":"$MentionDocTranslationInfo",
                    "pays":"$ActorGeo_CountryCode"}, 
                                                                                                                      #   // NE PAS FAIRE CA, EN GROUPANT PAR GLOBALEVENTID
            "countNegativeAvgTone" : { "$sum": "$lessThan0" }, 
            #Compte les AvgTone négatifs groupé par MonthYear, ActorCountryCode, MentionDocTranslationInfo
            "countPositiveAvgTone" : { "$sum": "$moreThan0" }
        }
    }]
    
    result_query = pd.DataFrame(list(collection.aggregate(pipeline, allowDiskUse = True)))
    DF = pd.DataFrame()
    
    DF['Date'] = pd.to_datetime(result_query.apply(lambda row : row["_id"]["mois"] , axis = 1), format='%Y%m')
    DF['Country'] = result_query.apply(lambda row : row["_id"]["pays"] , axis = 1)
    DF['Langue'] = result_query.apply(lambda row : row["_id"]["langue"] , axis = 1)
    DF['Actor'] = result_query.apply(lambda row : row["_id"]["actor"] , axis = 1)
    DF['#NegTone'] = result_query["countNegativeAvgTone"]
    DF['#PosTone'] = result_query["countPositiveAvgTone"]
    
    return DF

def Triplet_Q3(Month, Country, Langue, DF):
    
    return DF.loc[(DF["Country"]==Country) & (DF['Langue'] == Langue) & (DF['Date'].dt.month == Month)]


In [26]:
DF_Q3 = Function3()
DF_Q3.sort_values(by=["#NegTone"], ascending = False).head(10).style.hide_index()

Date,Country,Langue,Actor,#NegTone,#PosTone
2018-03-01 00:00:00,,ENG,,1738923,843218
2018-08-01 00:00:00,,ENG,,1664797,772089
2018-04-01 00:00:00,,ENG,,1655007,791863
2018-10-01 00:00:00,,ENG,,1643388,820491
2018-01-01 00:00:00,,ENG,,1606247,756223
2018-07-01 00:00:00,,ENG,,1571569,746287
2018-06-01 00:00:00,,ENG,,1565652,790609
2018-11-01 00:00:00,,ENG,,1559158,780907
2018-09-01 00:00:00,,ENG,,1529138,736368
2018-02-01 00:00:00,,ENG,,1524897,730159


In [34]:
DF_Q3.to_csv('DF_Q3.csv')

In [27]:
Triplet_Q3(2, "US", "ENG", DF_Q3)

,Date,Country,Langue,Actor,#NegTone,#PosTone
448,2008-02-01,US,ENG,ACTIVIST,6,0
449,2008-02-01,US,ENG,ACTOR,1,0
466,2008-02-01,US,ENG,AMERICAN,3,2
467,2008-02-01,US,ENG,APPEALS COURT,1,0
469,2008-02-01,US,ENG,ARIZONA,1,1
470,2008-02-01,US,ENG,ARMED FORCES,2,0
474,2008-02-01,US,ENG,AUSTIN,6,0
480,2008-02-01,US,ENG,AUSTRALIAN,0,1
489,2008-02-01,US,ENG,BANK,0,2
497,2008-02-01,US,ENG,BORDER PATROL,1,0


# Part 4.

find out which actors/countries/organisations are the most divisive (e.g. had a positive perception in one part of the world and a negative perception in the rest of the world). Allow one aggregation per day/month/year.

In [22]:
def Function4(periodicity, correspondance=1):
    
    if periodicity == "Day":
        typeDate = "$SQLDATE"
    else:
        typeDate = "$MonthYear"
    
    pipeline = [
   {"$group": {
       "_id" : { "periode": typeDate, "actor": "$ActorGeo_CountryCode"},
       "ToneStd": {"$stdDevPop": "$AvgTone"}
       }
       }
    ]
    
    result_query = pd.DataFrame(list(collection.aggregate(pipeline, allowDiskUse = True)))
    DF = pd.DataFrame()
    
    if typeDate == "Day":
        DF['Date'] = pd.to_datetime(result_query.apply(lambda row : row["_id"]["periode"] , axis = 1), format='%Y%m%d')
    else:
        DF['Date'] = pd.to_datetime(result_query.apply(lambda row : row["_id"]["periode"] , axis = 1), format='%Y%m')
    
    DF['Actor'] = result_query.apply(lambda row : row["_id"]["actor"] , axis = 1)
    DF['ToneSTD'] = result_query["ToneStd"]
    
    if correspondance==1:
        DF=replace_by_correspondence(DF, df_country_code, 'Actor')
    
    return DF

DF_Q4 = Function4("Month")
DF_Q4.sort_values(by=["Date"], ascending = False).head(10).style.hide_index()

Date,Actor,ToneSTD,Actor_corresp
2018-12-01 00:00:00,EG,4.5856,Egypt
2018-12-01 00:00:00,MZ,4.51344,Mozambique
2018-12-01 00:00:00,TZ,4.04808,Tanzania
2018-12-01 00:00:00,AN,2.61831,Andorra
2018-12-01 00:00:00,MR,3.6718,Mauritania
2018-12-01 00:00:00,WE,4.67616,Palestine
2018-12-01 00:00:00,KS,3.51484,Korea
2018-12-01 00:00:00,SH,5.41704,Saint Helena
2018-12-01 00:00:00,CA,3.54589,Canada
2018-12-01 00:00:00,CB,4.20067,Cambodia


In [36]:
DF_Q4.to_csv('DF_Q4.csv')

In [24]:
DF_Q4.sort_values(by=["ToneSTD"], ascending = False).head(30).style.hide_index()

Date,Actor,ToneSTD,Actor_corresp
2017-06-01 00:00:00,FM,9.70697,Micronesia
2018-05-01 00:00:00,ER,9.24233,Eritrea
2018-11-01 00:00:00,SB,8.70176,Saint Pierre and Miquelon
2017-07-01 00:00:00,NS,8.25509,Suriname
2008-10-01 00:00:00,NI,7.90421,Nigeria
2008-02-01 00:00:00,NZ,7.5836,New Zealand
2017-08-01 00:00:00,IV,7.22287,Cote Ivoire
2017-06-01 00:00:00,MO,7.06373,Morocco
2017-10-01 00:00:00,RE,6.91919,Reunion
2008-12-01 00:00:00,ID,6.85231,Indonesia


In [23]:
def Wrong_Function4():
    
    pipeline4 = [
    {"$project": {
        "ActorName": 1, "SQLDATE": 1, "AvgTone": 1,
        "hemisphereNordAvgTone": {"$cond": [{"$gt": ["$ActorGeo_Lat", 0]}, "$AvgTone", 0]},
        "hemisphereSudAvgTone": {"$cond": [{"$lt": ["$ActorGeo_Lat", 0]}, "$AvgTone", 0]}
        }
    },
    {"$group": {
        "_id": {"actor":"$ActorName", "date": "$SQLDATE"},
        "hemisphereNordAvgToneMean": {"$avg": "$hemisphereNordAvgTone"},
        "hemisphereSudAvgToneMean": {"$avg": "$hemisphereSudAvgTone"}
        }
    },
    {"$project": {
        "hemisphereNordAvgToneMean": 1, "hemisphereSudAvgToneMean": 1,
        "deltaEntreHemispheresNordSud": {"$abs": {"$subtract": ["$hemisphereNordAvgToneMean", "$hemisphereSudAvgToneMean"]}} #// Magnitude entre moyennes AvgTone des hemisphères Nord et Sud
            }
    }
    ]
    
    result_query4 = pd.DataFrame(list(collection.aggregate(pipeline4, allowDiskUse = True)))
    df4 = pd.DataFrame()
    
    df4['Date'] = pd.to_datetime(result_query4.apply(lambda row : row["_id"]["date"] , axis = 1), format='%Y%m%d')
    df4['Actor'] = result_query4.apply(lambda row : row["_id"]["actor"] , axis = 1)
    df4['DifferenceHemisphere'] = result_query4["deltaEntreHemispheresNordSud"]
    df4['North_Avg_Tone'] = result_query4["hemisphereNordAvgToneMean"]
    df4['South_Avg_Tone'] = result_query4["hemisphereSudAvgToneMean"]
    
    return df4

wrong_Q4 = Wrong_Function4()
wrong_Q4.sort_values(by=["DifferenceHemisphere"], ascending = False).head(10).style.hide_index()

Date,Actor,DifferenceHemisphere,North_Avg_Tone,South_Avg_Tone
2018-10-16 00:00:00,CONMAN,25.641,-25.641,0
2018-12-01 00:00:00,UKRAIN,25,-25,0
2018-06-20 00:00:00,CORINTH,24.1282,-24.1282,0
2018-10-04 00:00:00,TONGAN,23.4043,0,23.4043
2017-04-09 00:00:00,CANBERRA,23.2285,0,-23.2285
2018-07-13 00:00:00,KOLN,23.0216,0,-23.0216
2018-02-08 00:00:00,FEDERATED STATES OF MICRONESIA,22.9592,-22.9592,0
2017-04-25 00:00:00,CINCINNATI,22.7273,-22.7273,0
2017-04-15 00:00:00,MARYLAND,22.7273,-22.7273,0
2018-03-27 00:00:00,ESFAHAN,22.6415,-22.6415,0


In [37]:
wrong_Q4.to_csv('wrong_Q4.csv')

# Bonus

It will allow us to better understand the evolution of the relations between the different countries, by studying the tone of the mentions (positive/negative) in the media articles of each country.

In [12]:
def Function_Bonus_pt1(paysTest1, paysTest2, anneeTest):

    pipeline = [
    {"$match" :
       {"$or" :
           [
           {"$and": [{"ActorGeo_CountryCode": paysTest1},{"Year": anneeTest}] },
           {"$and": [{"ActorGeo_CountryCode": paysTest2},{"Year": anneeTest}] }
           ]
       }
    },
    {"$group" : {
       "_id" : {"fractionDate": "$FractionDate","globalEventId": "$GLOBALEVENTID", "actor": "$ActorGeo_CountryCode"},
       "avgToneMean": {"$avg": "$AvgTone"}
       }
    }        
    ]
    
    result_query = pd.DataFrame(list(collection.aggregate(pipeline, allowDiskUse = True)))
    
    DF = pd.DataFrame()
    
    DF['FracDate'] = result_query.apply(lambda row : row["_id"]["fractionDate"] , axis = 1)
    DF['EventID'] = result_query.apply(lambda row : row["_id"]["globalEventId"] , axis = 1)
    DF['AvgTone'] = result_query["avgToneMean"]
    DF['Actor'] = result_query.apply(lambda row : row["_id"]["actor"] , axis = 1)
    
    return DF

In [ ]:
DF_bon11 = Function_Bonus_pt1("FR", "EN", 2018)
DF_bon12 = Function_Bonus_pt1("CH", "RS", 2018)


frames = [DF_bon11, DF_bon12]

DF_bon111 = pd.concat(frames, axis=0)

In [ ]:
DF_bon1.groupby("FracDate")["AvgTone"].mean().plot()

In [ ]:
df_country = DF_bon111.groupby("Actor")["AvgTone"].mean()
countries = ["FR","EN","US","PL","CH","RS","KN"]
selected_country = df_country[countries]
ind = np.arange(len(selected_country))  # the x locations for the groups
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind - width/2, selected_country - df_country.mean(), width,
                color='SkyBlue', label='Country')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('AvgTone')
ax.set_title('AvgTone per country')
ax.set_xticks(ind)
ax.set_xticklabels(("FR","EN","US","PL","CH","RS","KN"))
ax.legend()
def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.
    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """
    xpos = xpos.lower()  # normalize the case of the parameter
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0.5, 'right': 0.57, 'left': 0.43}  # x_txt = x + w*off
    for rect in rects:
        height = rect.get_height()
autolabel(rects1, "left")
plt.show()

<strong>Not Running</strong>

In [ ]:
def Function_Bonus_pt2(moisTest):


    result_query = pd.DataFrame(list(collection.find( {"MonthYear":moisTest},
                                                   {"AvgTone":1, "ActionGeo_Lat":1, "ActionGeo_Long":1})))
    
    DF = pd.DataFrame()
    
    DF['AvgTone'] = result_query.apply(lambda row : row["AvgTone"] , axis = 1)
    DF['ActionGeo_Lat'] = result_query.apply(lambda row : row["ActionGeo_Lat"] , axis = 1)
    DF['ActionGeo_Long'] = result_query.apply(lambda row : row["ActionGeo_Long"] , axis = 1)
    
    return DF

In [ ]:
DF_bon2 = Function_Bonus_pt2(201801)
DF_bon2

In [ ]:
# base map 
np_bon2 = np.array(DF_bon2)

maxTone = np.max(DF_bon2["AvgTone"])
minTone = np.min(DF_bon2["AvgTone"])

def avgToColor(avgTone):
    red = 0.5 - avgTone/(-minTone)
    red = max(min(red,1),0)
    green = 0.5 + avgTone/(maxTone)
    green = max(min(green,1),0)
    return red, green

plt.figure(figsize=(20,20))
m = Basemap(projection='mill',lon_0=0,resolution='c')
m.drawcoastlines()
m.fillcontinents(color='gray',lake_color='white')
# draw parallels and meridians.
m.drawparallels(np.arange(-90.,120.,30.))
m.drawmeridians(np.arange(0.,420.,60.))
# draw a black dot at the center.

for avgTone, lat, lon in np_bon2[:10000]:
    xpt, ypt = m(lon, lat)
    red, green = avgToColor(avgTone)
    print(red, green)
    m.plot([xpt],[ypt], color=((red,green,0,1)))
    
# draw the title.
plt.title('Azimuthal Equidistant Projection')
plt.show()

In [ ]:
end = time. time()
print(end - start)

<strong>Should output :</strong>

<center><img src='./image.png'></center>